In [1]:
import numpy as np
from pycalphad import Model, Database, calculate, equilibrium
import pycalphad.variables as v

#dbf = Database('2016-08-10-AlGdMgand18RLPSO-for 3d plot.tdb')
dbf = Database('alfe_sei.TDB')
models = {key: Model(dbf, ['AL', 'FE', 'VA'], key) for key in dbf.phases.keys()}

In [2]:
#Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
%load_ext cython
from Cython.Compiler.Options import _directive_defaults

_directive_defaults['linetrace'] = True
_directive_defaults['binding'] = True

/home/rotis/anaconda/envs/calphadpy3/lib/python3.5/site-packages/Cython/Distutils/old_build_ext.py:30: UserWarning: Cython.Distutils.old_build_ext does not properly handle dependencies and is deprecated.
  "Cython.Distutils.old_build_ext does not properly handle dependencies "


In [28]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
import pycalphad.variables as v
from pycalphad.core.utils import unpack_kwarg
from pycalphad.core.utils import unpack_condition, unpack_phases
from pycalphad import calculate, Model
from pycalphad.constraints import mole_fraction
from pycalphad.core.lower_convex_hull import lower_convex_hull
from pycalphad.core.sympydiff_utils import build_functions as compiled_build_functions
from pycalphad.core.constants import MIN_SITE_FRACTION
from pycalphad.core.eqsolver import _solve_eq_at_conditions
from sympy import Add, Symbol
import dask
from dask import delayed
import dask.multiprocessing, dask.async
from xarray import Dataset
cimport numpy as np
import numpy as np
from collections import namedtuple, OrderedDict
from datetime import datetime
from pycalphad.core.phase_rec cimport PhaseRecord, obj, grad, hess
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
def _build_multiphase_system(int[:] phase_dof, phases, cur_conds, double[:] site_fracs, double[:] phase_fracs,
                              l_constraints, constraint_jac,
                              np.ndarray[ndim=3, dtype=np.float64_t] constraint_hess,
                              np.ndarray[ndim=1, dtype=np.float64_t] l_multipliers,
                              phase_records):
    cdef double[::1] obj_result =  np.empty(1)
    cdef double[::1] obj_res = np.empty(1)
    cdef int max_phase_dof = max(phase_dof)
    cdef double[::1] grad_res = np.empty(2+max_phase_dof)
    cdef double[::1,:] tmp_hess = np.empty((2+max_phase_dof, 2+max_phase_dof), order='F')
    cdef double* tmp_hess_ptr = &tmp_hess[0,0]
    cdef double[::1] dof = np.empty(2+max_phase_dof)
    cdef double[::1,:] dof_2d_view = <double[:1:1, :dof.shape[0]]>&dof[0]
    cdef int num_vars = len(site_fracs) + len(phases)
    cdef double[::1,:] l_hessian = np.zeros((num_vars, num_vars), order='F')
    cdef double[::1] gradient_term = np.zeros(num_vars)
    cdef int var_offset = 0
    cdef int phase_idx = 0
    cdef int cur_phase_dof, constraint_idx, dof_x_idx, dof_y_idx, hess_x, hess_y, hess_idx
    cdef double phase_frac
    cdef PhaseRecord prn
    dof[0] = cur_conds['P']
    dof[1] = cur_conds['T']

    for name, phase_frac in zip(phases, phase_fracs):
        prn = phase_records[name]
        with nogil:
            cur_phase_dof = <int>phase_dof[phase_idx]
            dof[2:2+cur_phase_dof] = site_fracs[var_offset:var_offset + cur_phase_dof]
            obj(prn, obj_res, dof_2d_view, 1)
            grad(prn, grad_res, dof)
            hess(prn, tmp_hess, dof)
            for dof_x_idx in range(cur_phase_dof):
                gradient_term[var_offset + dof_x_idx] = \
                    phase_frac * grad_res[2+dof_x_idx]  # Remove P,T grad part
            gradient_term[site_fracs.shape[0] + phase_idx] = obj_res[0]

            for dof_x_idx in range(cur_phase_dof):
                for dof_y_idx in range(dof_x_idx,cur_phase_dof):
                    l_hessian[var_offset+dof_x_idx, var_offset+dof_y_idx] = \
                      phase_frac * tmp_hess_ptr[2+dof_x_idx + (2+cur_phase_dof)*(2+dof_y_idx)]
                    l_hessian[var_offset+dof_y_idx, var_offset+dof_x_idx] = \
                      l_hessian[var_offset+dof_x_idx, var_offset+dof_y_idx]
                # Phase fraction / site fraction cross derivative
                l_hessian[site_fracs.shape[0] + phase_idx, var_offset + dof_x_idx] = \
                     grad_res[2+dof_x_idx] # Remove P,T grad part
                l_hessian[var_offset + dof_x_idx, site_fracs.shape[0] + phase_idx] = grad_res[2+dof_x_idx]
            var_offset += cur_phase_dof
            phase_idx += 1
    l_hessian -= np.einsum('i,ijk->jk', l_multipliers, constraint_hess, order='F')
    return np.asarray(l_hessian), np.asarray(gradient_term)

In [29]:
%load_ext line_profiler
%lprun -f _build_multiphase_system equilibrium(dbf, ['AL', 'FE', 'VA'], list(dbf.phases.keys()), {v.T: 700, v.X('AL'): (0,1,0.02), v.P: 101325}, model=models, build_multiphase_system=_build_multiphase_system)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
